In [ ]:
from pydantic import BaseModel, Field
from typing import Any


class PromptTokensDetails(BaseModel):
    """Details about token usage for the prompt."""

    audio_tokens: int | None = None
    """Count of audio tokens used in the prompt."""

    cached_tokens: int | None = None
    """Count of tokens served from cache."""


class Usage(BaseModel):
    """Information about token usage in the request and response."""

    completion_tokens: int
    """Number of tokens in the generated response."""

    prompt_tokens: int
    """Number of tokens in the prompt."""

    total_tokens: int
    """Total tokens used in the request (prompt + completion)."""

    completion_tokens_details: dict[str, Any] | None = None
    """Detailed information about completion token usage."""

    prompt_tokens_details: PromptTokensDetails | None = None
    """Detailed information about prompt token usage."""

    cache_creation_input_tokens: int | None = None
    """Tokens used for creating cache entries."""

    cache_read_input_tokens: int | None = None
    """Tokens read from cache."""

    service_tier: str | None = None
    """Service tier for the API call."""


class Function(BaseModel):
    """Represents a function to be called by a tool."""

    name: str
    """Name of the function to call."""

    arguments: str
    """JSON-formatted string containing the function arguments."""


class ToolCall(BaseModel):
    """Information about a tool call made by the model."""

    id: str
    """Unique identifier for the tool call."""

    type: str = "function"
    """Type of tool call, typically 'function'."""

    function: Function
    """Function information for the tool call."""

    index: int | None = None
    """Optional index for ordering multiple tool calls."""


class TextMessage(BaseModel):
    """Message containing text content generated by the model."""

    content: str
    """The text content of the message."""

    role: str
    """Role of the message sender, typically 'assistant'."""


class ToolCallMessage(BaseModel):
    """Message containing tool calls generated by the model."""

    content: None = None
    """Always None for tool call messages."""

    role: str
    """Role of the message sender, typically 'assistant'."""

    tool_calls: list[ToolCall]
    """List of tool calls the model wants to make."""


class Choice(BaseModel):
    """A single response option generated by the model."""

    finish_reason: str
    """Reason why the model stopped generating, e.g., 'stop', 'tool_calls'."""

    index: int
    """Index of this choice in the response."""

    message: TextMessage | ToolCallMessage
    """The content of this response choice, either text or tool calls."""


class Response(BaseModel):
    """Complete response from the Claude API."""

    id: str
    """Unique identifier for the API response."""

    created: int
    """Unix timestamp (in seconds) when the response was created."""

    model: str
    """The model that generated the response."""

    object: str = "chat.completion"
    """Object type, always 'chat.completion' for Claude chat responses."""

    system_fingerprint: str | None = None
    """Identifier for the system configuration used."""

    choices: list[Choice]
    """List of completion choices the model generated."""

    usage: Usage
    """Information about token usage for this request."""

    args: list[Any] = Field(default_factory=list)
    """Additional arguments used for the request."""

    kwargs: dict[str, Any] = Field(default_factory=dict)
    """Additional keyword arguments used for the request."""

    timestamp: float | None = None
    """Precise timestamp when the response was generated."""

    cost: float | None = None
    """Cost of the API call in USD."""

class ContentItem(BaseModel):
    """Individual content item within a message."""
    
    type: str
    """Type of content, typically 'text'."""
    
    text: str
    """The actual text content."""
    
    cache_control: dict[str, str] | None = None
    """Optional caching directives for this content item."""


class Message(BaseModel):
    """A message in the conversation sent to or received from the LLM."""
    
    content: list[ContentItem]
    """List of content items composing this message."""
    
    role: str
    """Role of the message sender (e.g., 'user', 'assistant', 'system')."""
    
    tool_calls: list[dict[str, Any]] | None = None
    """Tool calls made by the assistant, if any."""
    
    tool_call_id: str | None = None
    """ID of a tool call this message is responding to, if applicable."""
    
    name: str | None = None
    """Optional name identifier for the message sender."""


class Completion(BaseModel):
    """A single completion request-response pair."""
    
    messages: list[Message]
    """Messages sent to the LLM as the completion request."""
    
    response: Response
    """Response received from the LLM API."""


class Conversation(BaseModel):
    """A full conversation consisting of multiple completion turns."""
    
    turns: list[Completion]
    """Sequential completion request-response turns in the conversation."""

In [ ]:
from pydantic import BaseModel, Field
from typing import Any
from pathlib import Path

def conversation_paths_from_evaluation(evaluation_path: Path) -> list[Path]:
    return [
        path for path in (evaluation_path / "llm_completions").iterdir()
        if path.is_dir()
    ]

def load_conversation(completion_logs_path: Path) -> Conversation:
    # The completion logs path should point to a directory that contains individual
    # JSON files representing turns in the conversation. We need to stitch those
    # back together in order.
    turns = []
    for turn_path in completion_logs_path.iterdir():
        with open(turn_path, 'r') as f:
            turn = Completion.model_validate_json(f.read())
            turns.append(turn)
    return Conversation(turns=turns)

In [64]:
EVALUATION_PATH = "/Users/calvin/all-hands/OpenHands/evaluation/evaluation_outputs/outputs/princeton-nlp__SWE-bench_Verified-test/CodeActAgent/claude-3-7-sonnet-20250219_maxiter_100_N_v0.28.1-no-hint-run_1"

conversation_paths = conversation_paths_from_evaluation(Path(EVALUATION_PATH))
conversation = load_conversation(conversation_paths[0])

In [66]:
for completion in conversation.turns:
    print(completion.response)

{'id': 'chatcmpl-f2690c5f-5cd8-4b57-a771-82b74f8e7f55', 'created': 1741819768, 'model': 'litellm_proxy/claude-3-7-sonnet-20250219', 'object': 'chat.completion', 'system_fingerprint': None, 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'message': {'content': None, 'role': 'assistant', 'tool_calls': [{'index': 0, 'function': {'arguments': '{"command": "view", "path": "/workspace/scikit-learn__scikit-learn__0.21/sklearn/linear_model/least_angle.py", "view_range": [1100, 1200]}', 'name': 'str_replace_editor'}, 'id': 'toolu_01257TYjM7UmCp6aQ6RReaou', 'type': 'function'}], 'function_call': None}}], 'usage': {'completion_tokens': 126, 'prompt_tokens': 19249, 'total_tokens': 19375, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 17562}, 'cache_creation_input_tokens': 1683, 'cache_read_input_tokens': 17562}, 'service_tier': None}
{'id': 'chatcmpl-4bb53081-d6b8-4a4f-b8aa-2560fedd89a4', 'created': 1741819917, 'model': 'litellm_proxy/cl